<marquee scrollamount="20" direction="right"><h1 align="center">WINNING TEAM PREDICTOR</h1></marquee>

<h2 align="center">------------ ORGANIZING DATASET ------------</h2>

### Importing the Libaries

In [ ]:
import pandas as pd
import numpy as np
from colorama import Fore
pd.options.mode.chained_assignment = None  # otherwise gives unncessary warning for direct indexing

### Loading dataset into DataFrame

In [ ]:
try:
    delivery=pd.read_csv("deliveries.csv")
    matches=pd.read_csv("matches.csv")
except FileNotFoundError:
    print(Fore.RED+"Dataset Not Found!"+Fore.RESET)
else:
    print(Fore.GREEN+"Dataset Loaded Successfully"+Fore.BLACK)

### Checking the dataset and analyzing 

In [ ]:
matches.head()

In [ ]:
matches.shape  # no. of rows and no. of columns

In [ ]:
matches.groupby("Season")["Season"].count() # tells no. of matches played in every season

In [ ]:
matches.groupby("Season")["Season"].count().cumsum()  # cumulative sum 

In [ ]:
delivery.head()

In [ ]:
delivery.shape

### deciding which columns to consider for final dataframe

In [ ]:
matches.columns  # tells columns field --> we need to reduce it to make efficient in data analysis

In [ ]:
delivery.columns

In [ ]:
# important paramter for data analysis -->  batting team --> A , bowling team --> B , runs left B , balls left B , wickets left B , total runs scored by A , current run rate B , required run rate B , city , match result A or B 

### DATA CLEANING

In [ ]:
delivery.index

In [ ]:
matches.index

In [ ]:
delivery.head()

In [ ]:
# get total runs for each match in each inning
total_runs_df=delivery.groupby(["match_id","inning"])["total_runs"].sum()
total_runs_df=total_runs_df.reset_index()
total_runs_df.head(6)

In [ ]:
total_runs_df.shape

In [ ]:
# we require total runs of first team only
total_runs_df = total_runs_df.query("inning==1")
# total_runs_df.shape  # --> same rows count as matches , hence merge possible
total_runs_df

In [ ]:
# merging the total_runs scored by team1 with matches dataframe
match_df = pd.merge(matches,total_runs_df[["match_id","total_runs"]],left_on='id',right_on='match_id')

In [ ]:
# now this dataframe contains id two time therefor removing match_id
if("match_id" in match_df.columns):
    match_df = match_df.drop(columns="match_id")
    print("Dropped Successfully!")
match_df

In [ ]:
# now need to drop those teams who is not playing ipl currently
# all teams 
# match_df.groupby("team1")["team1"].count()
match_df["team1"].unique()   # Gujarat Lions ,Rising Pune Supergiants ,Kochi Tuskers Kerala ,Pune Warriors --> no longer playing
# Delhi Daredevils == Delhi Capitals
# Deccan Chargers == Sunrisers Hyderabad --> we will change the name

#### Modifying team name

In [ ]:
# modifing team name
match_df["team1"]=match_df["team1"].str.replace("Delhi Daredevils","Delhi Capitals")
match_df["team1"]=match_df["team1"].str.replace("Deccan Chargers","Sunrisers Hyderabad")

In [ ]:
# in team2
match_df["team2"]=match_df["team2"].str.replace("Delhi Daredevils","Delhi Capitals")
match_df["team2"]=match_df["team2"].str.replace("Deccan Chargers","Sunrisers Hyderabad") 

In [ ]:
# in toss winner
match_df["toss_winner"]=match_df["toss_winner"].str.replace("Delhi Daredevils","Delhi Capitals")
match_df["toss_winner"]=match_df["toss_winner"].str.replace("Deccan Chargers","Sunrisers Hyderabad")

In [ ]:
# in winner
match_df["winner"]=match_df["winner"].str.replace("Delhi Daredevils","Delhi Capitals")
match_df["winner"]=match_df["winner"].str.replace("Deccan Chargers","Sunrisers Hyderabad")

In [ ]:
# for delivery
delivery["batting_team"]=delivery["batting_team"].str.replace("Delhi Daredevils","Delhi Capitals")
delivery["batting_team"]=delivery["batting_team"].str.replace("Deccan Chargers","Sunrisers Hyderabad")

In [ ]:
delivery["bowling_team"]=delivery["bowling_team"].str.replace("Delhi Daredevils","Delhi Capitals")
delivery["bowling_team"]=delivery["bowling_team"].str.replace("Deccan Chargers","Sunrisers Hyderabad")

In [ ]:
match_df["team1"].value_counts()

#### Selecting Only those teams which consistently playing

In [ ]:
consistent_teams=["Royal Challengers Bangalore","Kolkata Knight Riders","Kings XI Punjab","Sunrisers Hyderabad"
                   ,"Mumbai Indians","Delhi Capitals","Chennai Super Kings","Rajasthan Royals"]

In [ ]:
match_df = match_df[(match_df["team1"].isin(consistent_teams)) & (match_df["team2"].isin(consistent_teams))]

In [ ]:
# match_df
match_df.shape  # no. of matches reduced from 756 to 641

#### removing those matches where match remain suspended due to rain and duckworth-lewis applied

In [ ]:
match_df["dl_applied"].value_counts()  # 15 matches were affected by rain --> hence removing it

In [ ]:
match_df = match_df[match_df["dl_applied"]==0]  # removed 15 matches
match_df

#### Retrieving Those Columns which are required from match_df DataFrame

In [ ]:
match_df.columns

In [ ]:
# id --> to join with delivery dataframe , city , winner , total_runs
match_df = match_df[["id","city","winner","total_runs"]]
match_df

In [ ]:
delivery.shape

In [ ]:
delivery

In [ ]:
# now merging match_df with delivery --> where id is same --> each match has unique id
merge_df = pd.merge(delivery,match_df,left_on="match_id",right_on="id")
merge_df

In [ ]:
# now removing id as it contains 2 times
if("id" in merge_df.columns):
    merge_df.drop(columns="id",inplace=True)
merge_df

In [ ]:
# here total_runs_y means target for 2nd team_x
merge_df.rename(columns={"total_runs_y":"target"},inplace=True)
merge_df

In [ ]:
# now incrementing target field by 1 as target is always +1 to runs scored by 1st team
merge_df["target"] = merge_df["target"].apply(lambda x:x+1)

In [ ]:
merge_df

In [ ]:
# now we need to predict on basis of target by 1st team --> hence removing innnings = 1  (since we already obtained the target) and ball to ball delivery is not required
merge_df = merge_df[merge_df["inning"]==2]
merge_df

#### now calculating balls left , runs left and wickets left after each ball

#### <u>for balls left</u> --> 120 - ((over-1)*6 + ball)

In [ ]:
# adding new column --> balls faced
merge_df["balls_faced"] = (merge_df["over"]-1)*6 +(merge_df["ball"])
merge_df

In [ ]:
# adding new column --> balls_left
merge_df["balls_left"] = 120 - merge_df["balls_faced"]
merge_df

#### <u>for runs left</u> for each match --> first groupby by match_id and then find cumulative sum , then minus from target 

In [ ]:
# adding new column --> score
merge_df["current_score"] = merge_df.groupby("match_id")["total_runs_x"].cumsum()
merge_df

In [ ]:
# adding new column --> runs_left
merge_df["runs_left"] = merge_df["target"] - merge_df["current_score"]
merge_df

In [ ]:
merge_df

#### <u>for wickets left</u> --> add 1 when player_dismissed == 1 and (10 - answer)

In [ ]:
merge_df["player_dismissed"].fillna(0,inplace=True)
merge_df["player_dismissed"] = merge_df["player_dismissed"].apply(lambda x:x if x==0 else 1)  # if not 0 then replace by 1 and 1 means wicket fall

In [ ]:
# adding new column --> wickets_fallen
merge_df["wickets_fallen"] = merge_df.groupby("match_id")["player_dismissed"].cumsum()
merge_df

In [ ]:
# adding new column --> wickets_left
merge_df["wickets_left"] = 10 - merge_df["wickets_fallen"]
merge_df

In [ ]:
# merge_df.query("match_id==1") --> to check

#### calculating current run rate and required run rate

#### CRR --> (current_score) * 6 / (balls_faced)

In [ ]:
# adding new column --> CRR
merge_df["CRR"] = merge_df["current_score"] * 6 / (merge_df["balls_faced"])
merge_df

#### RRR -->  (runs_left * 6) / (balls_left)

In [ ]:
# adding new column --> RRR
merge_df["RRR"] = merge_df["runs_left"] * 6 / (merge_df["balls_left"])
merge_df

#### getting result in terms of 0 and 1  --->  0 means team1 (first batting) won and 1 means team2 (chasing team) won , result will be compare on basis of chasing team 

In [ ]:
# merge_df["batting_team"].apply(lambda x:x if x=="Royal Challengers Bangalore" else 1) --> lambda takes 1D value . hence cannot use , instead use function
def final_result(match):
    if(match["batting_team"]==match["winner"]):
        return 1
    else:
        return 0
merge_df["result"] = merge_df.apply(final_result,axis=1) 

In [ ]:
merge_df["batting_team"].value_counts()

### NOW ALL THE INFORMATION IS COMPUTED AND ANALYZED , NOW RETRIEVING THOSE FIELDS ONLY WHICH ARE REQUIRED FOR CREATING PREDICTION MODEL 

batting team --> A , bowling team --> B , runs left B , balls left B , wickets left B , total runs scored by A , current run rate B , required run rate B , city , match result A or B 

In [ ]:
final_data = merge_df[["match_id","city","batting_team","bowling_team",
        "target","current_score","runs_left",
        "balls_faced","balls_left",
        "wickets_fallen","wickets_left",
        "CRR","RRR","result"]]

In [ ]:
print(final_data.shape)
final_data  # here result gives answer about whether the chasing team won or not

In [ ]:
# checking whether sum cells are empty
final_data.isnull().sum()  # city contains null values --> removing them

In [ ]:
final_data = final_data.dropna()
print(final_data.shape)
final_data

In [ ]:
# since rrr was becoming -inf or inf when balls_left becomes 0 , hence removing last ball --> it doesn't affects much on model
final_data = final_data[final_data["balls_left"] != 0]   

In [ ]:
final_data.shape

In [ ]:
final_data = final_data.reset_index()  #--> reseted the index 
final_data.drop(columns=["index"],axis=1,inplace=True)

In [ ]:
final_data = final_data.sample(frac=1)    # shuffling the dataframe so, the ml model can divide and result doesn't look baised

In [ ]:
final_data.sample()

### Saving the dataFrame

In [ ]:
final_data.to_csv("final_data.csv",index=False)

### NOW BUILDING MODEL USING SKLEARN ( SCIKIT LEARN - MACHINE LEARNING LIBRARY)

In [ ]:
# on the basis of runs, balls, wicket, city, battingteam , bowling team and target we are going to predict the winning ratio of both teams (result in percentage) --> result is dependent variable (response variable) and all other paramters are independent variables (predictor variable) 

In [ ]:
final_data

##### Train Test Split : https://www.youtube.com/watch?v=fwY9Qv96DJY

<img src="maxresdefault.jpg" alt="traintestsplit" width="700px">

In [ ]:
from sklearn.model_selection import train_test_split 
# split the dataframe into training and testing phase
# to train the model using training sample (80% mostly)  --> selects random sample
# test the model using test sample

# firstly dividing the independent and dependent variable in X and y on which testing and training required
 
# X  --> independent varaiable
X = final_data.iloc[:,1:-1]   # all atributes except last column
X

In [ ]:
# Y --> dependent variable
y = final_data.iloc[:,-1]  # only last column (result)
y

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2 , random_state=1)  # test size -> 20% and train size --> 80%

In [ ]:
X_train    # here city , batting team and bowling team is in string hence need to use one hot encoding otherwise gives problem since ml models are good in managing numerical data

##### Column Transformer : https://www.youtube.com/watch?v=5TVj6iEBR4I
##### One Hot Encoding : https://www.youtube.com/watch?v=9yl6-HEY7_s

<img src="onehotencoding.png" alt="ohe" width=700>

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# differnet encoders --> SimpleImputer --> replaces null/NaN with mean values  , OneHotDecoder , OridinalEncoder
# since here categorical data is in 3 columns so, need to use column transformer

ohe = ColumnTransformer(transformers=[("trf",OneHotEncoder(sparse=False,drop="first"),["city","batting_team","bowling_team"])],remainder="passthrough")  # remainder --> drop or remain as it is (passthrough) the other columns 


In [ ]:
# pipelining and logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline(
    steps=[("step1",ohe),("step2",LogisticRegression(solver="liblinear"))]
)

##### fit , transform , fit_transform , predict : https://www.youtube.com/watch?v=BotYLBQfd5M 

fit --> it gives the required paramters for example in SimpleImputer --> Mean <br>
transform --> it transforms the data based on fit value ,sets null/NaN value to mean in SimpleImputer<br>
fit_transform --> combination of above 2 --> same as fit().transform()<br>
**fit and transform for train data**<br>
**transform for test data**<br>

in model only 2 process  --> fit and predict <br>
**fit for training data and predict for test data** <br>

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
predict_y = pipe.predict(X_test)   # preedicts based on trained values
predict_y

##### We calculate accuracy by dividing the number of correct predictions (the corresponding diagonal in the matrix) by the total number of samples. 

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(predict_y,y_test)
print("Accuracy : {:.3f} %".format(acc*100))  # can use other models too , but high accuracy doesn't mean it is suitable for our problem  --> since it doesn't give more flexibity

In [ ]:
# predict probability
pipe.predict_proba(X_test)

In [ ]:
import random
rand = list(final_data["match_id"])
match_id = random.choice(rand)

In [ ]:
final_data

In [ ]:
import pickle
with open("final_df.pkl","wb") as f:
    pickle.dump(final_data,f)

In [ ]:
# calculating the probability after each over in particular match
def match_prob(id,pipe):
    data = final_data[final_data["match_id"]==id]
    xyz = final_data.query(f"((balls_faced>=114 and balls_faced<=119) or balls_faced%6 ==0 ) and match_id == {id}")
    xyz.sort_values("balls_faced",inplace=True)
    wxy = xyz[xyz["balls_faced"]%6==0]
    wxy = wxy.append(xyz.iloc[-1])
    data = wxy.iloc[:,1:-1]
    result = pipe.predict_proba(data)
    data["lose_prob"] = np.round(result.T[0]*100,1)
    data["win_prob"] = np.round(result.T[1]*100,1)
    target = data["target"].values[0]
    print("Target-",target)
    new = data[["current_score","balls_faced","wickets_fallen","lose_prob","win_prob"]]
    new['end_of_over'] = range(1,new.shape[0]+1)
    new["runs_in_over"] = new["current_score"].diff()
    over1 = new["current_score"].values[0]
    wicket1 = new["wickets_fallen"].values[0]
    new["runs_in_over"] = new["runs_in_over"].fillna(over1) 
    new["runs_in_over"] = new["runs_in_over"].astype(int)
    new["wicket_in_over"] = new["wickets_fallen"].diff()
    new["wicket_in_over"] = new["wicket_in_over"].fillna(wicket1)
    new["wicket_in_over"] = new["wicket_in_over"].astype(int)
    return new,target

In [ ]:
X_train

In [ ]:
a , b = match_prob(69,pipe)
a

##### testing

In [ ]:
new_df,target = match_prob(1,pipe)
new_df.reset_index(inplace=True)
new_df.drop(columns=["index"],axis=1)

In [ ]:
# visualiation 
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(15,7))
x_value = new_df['end_of_over']
plt.plot(x_value,new_df['wicket_in_over'],color='yellow',lw=3,label="Fall of wicket")
plt.plot(x_value,new_df['win_prob'],color='#00a65a',lw=3,label="Winning Ratio")
plt.plot(x_value,new_df['lose_prob'],color='red',lw=3,label="Losing Ratio")
plt.bar(x_value,new_df['runs_in_over'])
plt.xticks(np.arange(1,21))
plt.yticks(np.arange(0,101,20))
plt.title('Target-' + str(target))
plt.xlabel("End Of The Over")
plt.ylabel("Probabilty")
plt.legend()
plt.show()

In [ ]:
consistent_teams.sort()
consistent_teams

In [ ]:
city = list(final_data["city"].unique())
city.sort()
city

In [ ]:
import pickle
with open("Teams.pkl","wb") as f:
    pickle.dump(consistent_teams,f)  # storing structure in file

In [ ]:
import pickle
with open("Cities.pkl","wb") as f:
    pickle.dump(city,f)

*Pickle in Python is primarily used in serializing and deserializing a Python object structure. In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program state across sessions, or transport data over the network.* <br>
*"Pickling" is process which enables storage and preservation.*

**dump --> to store object structure in some file in binary(bytes format) <br>
load --> to load data from that file object in orginal data type object(list,dictionary,etc)**

In [ ]:
import pickle
with open("struct.pkl","wb") as f:
    pickle.dump(pipe,f)   # takes 2 parameters --> object structure and file object